<a href="https://colab.research.google.com/github/kishankc123/Advanced_Machine_Learning/blob/main/MovieLens100k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv torch_geometric -f https://data.pyg.org/whl/torch-2.3.0+cpu.html


Looking in links: https://data.pyg.org/whl/torch-2.3.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 513.6/513.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.8/783.8 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.2/217.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.2 MB/s eta 0:00:00


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip

--2025-12-06 04:14:42--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.96.204
Connecting to files.grouplens.org (files.grouplens.org)|128.101.96.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  16.3MB/s    in 0.3s    

2025-12-06 04:14:42 (16.3 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]



In [ ]:
!unzip ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


In [ ]:

# Read the CSV file using pandas
import pandas as pd

column_names = ["user_id","item_id" , "rating" , "timestamp"]

try:
    df = pd.read_csv('/content/ml-100k/u.data', sep='\t', header = None, names = column_names)
    print("Data successfully loaded:")
    print(df.head())

except FileNotFoundError:
    print(f"Error: File not found at {'/content/ml-100k/u.data'}")
except Exception as e:
    print(f"An error occurred: {e}")


Data successfully loaded:
   user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596


In [ ]:

# # Read the CSV file using pandas
# import pandas as pd

# # column_names = ["user_id","item_id" , "rating" , "timestamp"]

# try:
#     df = pd.read_csv('/content/ml-100k/u.data')
#     print("Data successfully loaded:")
#     print(df)

# except FileNotFoundError:
#     print(f"Error: File not found at {'/content/ml-100k/u.item'}")
# except Exception as e:
#     print(f"An error occurred: {e}")

In [ ]:
num_items = None

with open('/content/ml-100k/u.info', 'r') as f:
    for line in f:
        if "items" in line:
            num_items = int(line.split()[0])

print("Number of items:", num_items)


Number of items: 1682


In [ ]:
num_users = None

with open('/content/ml-100k/u.info', 'r') as f:
    for line in f:
        if "users" in line:
            num_users = int(line.split()[0])

print("Number of Users:", num_users)


Number of Users: 943


In [ ]:
num_ratings = None

with open('/content/ml-100k/u.info', 'r') as f:
    for line in f:
        if "ratings" in line:
            num_ratings = int(line.split()[0])

print("Number of Ratings:", num_ratings)


Number of Ratings: 100000


In [ ]:
import pandas as pd

movies = pd.read_csv(
    "/content/ml-100k/u.item",
    sep="|",
    header=None,
    encoding="latin-1",  # avoids issues with special characters
    names=[
        "movie_id", "title", "release_date", "video_release_date", "imdb_url",
        "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy",
        "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
        "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
    ],
)

genre_cols = [
    "Action", "Adventure", "Animation", "Children's", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

print(movies.head(1436))


      movie_id                           title release_date  \
0            1                Toy Story (1995)  01-Jan-1995   
1            2                GoldenEye (1995)  01-Jan-1995   
2            3               Four Rooms (1995)  01-Jan-1995   
3            4               Get Shorty (1995)  01-Jan-1995   
4            5                  Copycat (1995)  01-Jan-1995   
...        ...                             ...          ...   
1431      1432              Mighty, The (1998)  09-Oct-1998   
1432      1433             Men of Means (1998)  01-Jan-1997   
1433      1434            Shooting Fish (1997)  16-Jan-1998   
1434      1435  Steal Big, Steal Little (1995)  01-Jan-1995   
1435      1436                Mr. Jones (1993)  01-Jan-1993   

      video_release_date                                           imdb_url  \
0                    NaN  http://us.imdb.com/M/title-exact?Toy%20Story%2...   
1                    NaN  http://us.imdb.com/M/title-exact?GoldenEye%20(...   
2     

In [ ]:
#helper function to get the genre of movies
def get_movie_genres(row):
    return [g for g in genre_cols if row[g] == 1]

movies["genres"] = movies.apply(get_movie_genres, axis=1)

movie_id_to_genres = dict(zip(movies["movie_id"], movies["genres"]))


In [ ]:
num_nodes = num_users + num_items
print(num_nodes)


2625


In [ ]:
from sklearn.model_selection import train_test_split


# Train-test split (80/20), shuffle before splitting
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,   # for reproducibility
    shuffle=True
)

# Reset index (optional but cleaner)
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

print("Train size:", len(train_df))
print("Test size:", len(test_df))

Train size: 80000
Test size: 20000


In [ ]:
# from collections import defaultdict

# strong_genres = {}  # user_id -> set of strongly preferred genres

# # group ratings per user
# for user_id, user_rows in train_df.groupby("user_id"):
#     # sort by rating desc, then timestamp desc as tie-breaker
#     user_rows_sorted = user_rows.sort_values(
#         by=["rating", "timestamp"], ascending=[False, False]
#     )

#     # take top 5 rated movies
#     top5 = user_rows_sorted.head(5)

#     # count genres in these top 5
#     genre_count = defaultdict(int)

#     for row in top5.itertuples():
#         movie_id = row.item_id
#         genres = movie_id_to_genres.get(movie_id, [])
#         # count per-movie, not per-genre label occurrence
#         # (i.e., each movie contributes 1 to each of its genres)
#         for g in genres:
#             genre_count[g] += 1

#     # pick genres that appear in at least 2 of the top 5
#     strong = {g for g, c in genre_count.items() if c >= 2}
#     strong_genres[user_id] = strong


In [ ]:
import torch

# Convert user_id and item_id columns to tensors
train_user_ids = torch.tensor(train_df["user_id"].values, dtype=torch.long)
train_item_ids = torch.tensor(train_df["item_id"].values, dtype=torch.long)

# Map to node indices in a single shared space
train_user_nodes = train_user_ids - 1                     # 1..943 → 0..942
train_item_nodes= train_item_ids - 1 + num_users         # 1..1682 → 943..2624

print(train_user_nodes[:5])
print(train_item_nodes[:5])

src_index = torch.cat([train_user_nodes, train_item_nodes], dim=0)
dst_index = torch.cat([train_item_nodes, train_user_nodes], dim=0)

edge_index = torch.stack([src_index, dst_index], dim=0) #build bipartite graph

print("edge_index shape:", edge_index.shape)  # should be [2, 2 * num_interactions]


tensor([806, 473, 462, 138, 620])
tensor([2353, 1601, 1210, 1228, 1693])
edge_index shape: torch.Size([2, 160000])


In [ ]:
import torch

# 1. Number of interactions
num_interactions = len(train_df)
print("Number of interactions",num_interactions)

# 2. All user node IDs (one per row in train_df)
user_nodes_all = torch.tensor(train_df["user_id"].values, dtype=torch.long) - 1

# 3. All item node IDs (one per row in train_df)
item_nodes_all = torch.tensor(train_df["item_id"].values, dtype=torch.long) - 1 + num_users

print(user_nodes_all.shape)

Number of interactions 80000
torch.Size([80000])


Compute per-user genre counts (from train_df only)

In [ ]:
from collections import defaultdict

user_genre_counts = defaultdict(lambda: defaultdict(int))

for row in train_df.itertuples():
    user_id = row.user_id
    movie_id = row.item_id
    genres = movie_id_to_genres[movie_id]
    for g in genres:
        user_genre_counts[user_id][g] += 1


Define “liked genres” per user (≥ 3 movies)

In [ ]:
# user_liked_genres = {}

# for user_id, counts in user_genre_counts.items():
#     liked = {g for g, c in counts.items() if c >= 2}
#     user_liked_genres[user_id] = liked


In [ ]:
# age_group_to_genres = {
#     "teen": {
#         "Action", "Adventure", "Fantasy", "Animation", "Sci-Fi", "Children's"
#     },
#     "young_adult": {
#         "Action", "Sci-Fi", "Horror", "Comedy", "Adventure"
#     },
#     "adult": {
#         "Drama", "Action", "Thriller", "Crime", "Comedy"
#     },
#     "middle_age": {
#         "Drama", "Crime", "Thriller", "Documentary", "War"
#     },
#     "senior": {
#         "Drama", "Documentary", "War", "Western", "Romance"
#     },
# }


In [ ]:
# user_id_to_age_group = {
#     0: "teen",
#     1: "young_adult",
#     2: "adult",
#     3: "middle_age",
#     4: "senior"
# }


In [ ]:
# import torch

# num_interactions = len(train_df)

# # 1. Base weight from rating (you can adjust this if you use a different scheme)
# train_ratings = torch.tensor(train_df["rating"].values, dtype=torch.float)
# base_weight = train_ratings / 5.0   # [num_interactions]

# # 2. Start interaction_weight from base_weight
# interaction_weight = base_weight.clone()

# # 3. Apply mild genre bias, strong genre bias, and age bias in one pass
# for i, row in enumerate(train_df.itertuples()):
#     user_id = row.user_id
#     movie_id = row.item_id

#     # Start from whatever weight we already have (base + any earlier modifications)
#     w = interaction_weight[i]

#     # ----- Get movie genres -----
#     movie_genres = movie_id_to_genres.get(movie_id, [])

#     # ============================
#     #  A) MILD GENRE BIAS (5–15%)
#     # ============================
#     # user_liked_genres[user_id] = set of genres with >=3 movies
#     liked_genres = user_liked_genres.get(user_id, set())

#     # find any genre on this movie that the user mildly likes
#     mild_matches = [g for g in movie_genres if g in liked_genres]

#     if mild_matches:
#         # pick one genre to determine strength
#         g = mild_matches[0]
#         count = user_genre_counts[user_id][g]  # how many movies in this genre

#         if 3 <= count < 6:
#             w *= 1.05   # +5%
#         elif 6 <= count < 10:
#             w *= 1.10   # +10%
#         elif count >= 10:
#             w *= 1.15   # +15%

#     # ===============================
#     #  B) STRONG GENRE BIAS (+30%)
#     # ===============================
#     # strong_genres[user_id] = genres that appear >=2 times in top-5 rated
#     strong = strong_genres.get(user_id, set())
#     if any(g in strong for g in movie_genres):
#         w *= 1.30   # +30% strong bias

#     # ========================
#     #  C) AGE-GROUP BIAS (+20%)
#     # ========================
#     age_group = user_id_to_age_group.get(user_id, None)
#     if age_group is not None:
#         preferred_genres = age_group_to_genres.get(age_group, set())
#         if any(g in preferred_genres for g in movie_genres):
#             w *= 1.20   # +20% age-group bias

#     # write back the final weight for this interaction
#     interaction_weight[i] = w

# # 4. Build edge_weight for LightGCN (undirected graph: user->item, item->user)
# edge_weight = torch.cat([interaction_weight, interaction_weight], dim=0)

# print("interaction_weight shape:", interaction_weight.shape)  # [num_interactions]
# print("edge_weight shape:", edge_weight.shape)                # [2 * num_interactions]


NameError: name 'user_id_to_age_group' is not defined

In [ ]:
import torch

train_ratings = torch.tensor(train_df["rating"].values, dtype=torch.float)  # shape [num_interactions]
base_weight = train_ratings / 5.0



In [ ]:
# bias_factors = []

# for row, bw in zip(train_df.itertuples(), base_weight):
#     user_id = row.user_id
#     movie_id = row.item_id
#     genres = movie_id_to_genres[movie_id]
#     liked = user_liked_genres.get(user_id, set())

#     # any overlap between movie genres and user's liked genres?
#     match_genres = [g for g in genres if g in liked]

#     if not match_genres:
#         bias_factors.append(1.0)  # no boost
#         continue

#     # choose one genre to measure strength (e.g. first match)
#     g = match_genres[0]
#     count = user_genre_counts[user_id][g]

#     if 3 <= count < 6:
#         boost = 0.10
#     elif 6 <= count < 10:
#         boost = 0.15
#     else:  # >= 10
#         boost = 0.20

#     bias_factors.append(1.0 + boost)

# bias_factors = torch.tensor(bias_factors, dtype=torch.float)
# interaction_weight = base_weight * bias_factors  # [num_interactions]


In [ ]:
# for each row in train_df:
#     user_id = row.user_id
#     movie_id = row.item_id

#     w = base_weight[i]   # already possibly modified by mild/strong genre bias

#     # 1) Get user age group
#     age_group = user_id_to_age_group[user_id]

#     # 2) Get age-preferred genres
#     preferred_genres = age_group_to_genres.get(age_group, set())

#     # 3) Get this movie's genres
#     movie_genres = movie_id_to_genres[movie_id]

#     # 4) If overlap between movie genres and age-preferred genres → +20%
#     if any(g in preferred_genres for g in movie_genres):
#         w *= 1.2

#     interaction_weight[i] = w


In [ ]:
# # 2. Apply strong genre bias
# biased_weights = []

# for row, bw in zip(train_df.itertuples(), base_weight):
#     user_id = row.user_id
#     movie_id = row.item_id

#     movie_genres = movie_id_to_genres.get(movie_id, [])
#     strong = strong_genres.get(user_id, set())

#     # check if this movie intersects any strong genre
#     if any(g in strong for g in movie_genres):
#         bw = bw * 1.3  # +30% strong bias

#     biased_weights.append(bw)

# interaction_weight = torch.stack(biased_weights)  # [num_interactions]


In [ ]:
#interaction_weight = train_ratings / 5.0   # shape [num_interactions]
edge_weight = torch.cat([base_weight, base_weight], dim=0)  # shape [2 * num_interactions]
print(edge_weight.size())


torch.Size([160000])


In [ ]:
from torch_geometric.nn import LightGCN

model = LightGCN(
    num_nodes=num_nodes,
    embedding_dim=128,
    num_layers=3,
)

/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/libpyg.so
  import torch_geometric.typing
/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_scatter/_version_cpu.so
  import torch_geometric.typing
/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: Could not load this library: /usr/local/lib/python3.12/dist-packages/torch_cluster/_version_cpu.so
  import torch_geometric.typing
/usr/local/lib/python3.12/dist-packages/torch_geometric/__init__.py:4: UserWarning: An issue occurred while importing 'torch-spli

In [ ]:
# # Prepare positive interactions for BPR(same thing as below)
# user_tensor = torch.tensor(train_df['user_id'].values) - 1
# item_tensor = torch.tensor(train_df['item_id'].values) - 1 + num_users

In [ ]:
print("num_users:", num_users)
print("num_items:", num_items)
print("num_nodes:", num_nodes)
print("edge_index shape:", edge_index.shape)
print("src_index shape:", train_user_ids.shape)
print("dst_index shape:", train_item_ids.shape)


num_users: 943
num_items: 1682
num_nodes: 2625
edge_index shape: torch.Size([2, 160000])
src_index shape: torch.Size([80000])
dst_index shape: torch.Size([80000])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
batch_size = 1024

for epoch in range(500):
    model.train()

    # sample a batch of positive interactions
    idx = torch.randint(0, num_interactions, (batch_size,))
    users_batch = user_nodes_all[idx]
    pos_items_batch = item_nodes_all[idx]

    # sample negative items (any item node index)
    neg_items_batch = torch.randint(
        low=num_users,          # first item node index
        high=num_nodes,         # last + 1
        size=(batch_size,),
    )

    # Build edge_label_index for pos & neg
    pos_edge_label_index = torch.stack(
        [users_batch, pos_items_batch], dim=0
    )
    neg_edge_label_index = torch.stack(
        [users_batch, neg_items_batch], dim=0
    )

    optimizer.zero_grad()

    # Get scores from LightGCN correctly:
    pos_score = model(
        edge_index,
        edge_weight=edge_weight,
        edge_label_index=pos_edge_label_index)

    neg_score = model(
        edge_index,
        edge_weight=edge_weight,
        edge_label_index=neg_edge_label_index)


    # BPR loss
    loss = model.recommendation_loss(
        pos_score,
        neg_score,
        node_id=users_batch,
    )

    loss.backward()
    optimizer.step()

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, loss = {loss.item():.4f}")


Epoch 0, loss = 0.6931
Epoch 5, loss = 0.6931
Epoch 10, loss = 0.6931
Epoch 15, loss = 0.6930
Epoch 20, loss = 0.6929
Epoch 25, loss = 0.6927
Epoch 30, loss = 0.6922
Epoch 35, loss = 0.6910
Epoch 40, loss = 0.6888
Epoch 45, loss = 0.6847
Epoch 50, loss = 0.6797
Epoch 55, loss = 0.6710
Epoch 60, loss = 0.6604
Epoch 65, loss = 0.6491
Epoch 70, loss = 0.6315
Epoch 75, loss = 0.6155
Epoch 80, loss = 0.5857
Epoch 85, loss = 0.5766
Epoch 90, loss = 0.5531
Epoch 95, loss = 0.5508
Epoch 100, loss = 0.5218
Epoch 105, loss = 0.5221
Epoch 110, loss = 0.5037
Epoch 115, loss = 0.4859
Epoch 120, loss = 0.4898
Epoch 125, loss = 0.4771
Epoch 130, loss = 0.4838
Epoch 135, loss = 0.4809
Epoch 140, loss = 0.4833
Epoch 145, loss = 0.4808
Epoch 150, loss = 0.4851
Epoch 155, loss = 0.4725
Epoch 160, loss = 0.4619
Epoch 165, loss = 0.4443
Epoch 170, loss = 0.4657
Epoch 175, loss = 0.4660
Epoch 180, loss = 0.4805
Epoch 185, loss = 0.4693
Epoch 190, loss = 0.4386
Epoch 195, loss = 0.4717
Epoch 200, loss = 0.47

We need to train the model and then recommend the movies.

In [ ]:

model = model.to(device)
edge_index = edge_index.to(device)

src = torch.arange(num_users, device=device)            # user nodes: 0..942
dst = torch.arange(num_users, num_nodes, device=device) # item nodes: 943..2624

print("src shape:", src.shape)   # torch.Size([943])
print("dst shape:", dst.shape)   # torch.Size([1682])


src shape: torch.Size([943])
dst shape: torch.Size([1682])


In [ ]:
model.eval()
recommended_items = model.recommend(
    edge_index=edge_index,
    edge_weight=edge_weight,
    src_index = src,
    dst_index = dst,
    k=10
)

# print(recommended_items)

In [ ]:
mapped_recommendations = []

for i in range(recommended_items.size(0)):       # loop over users
    user_recommendations = []

    for node in recommended_items[i]:            # loop over recommended nodes for this user
        item_id = node.item() - num_users + 1
        user_recommendations.append(item_id)

    mapped_recommendations.append(user_recommendations)

print(mapped_recommendations)



[[50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 181, 258, 127, 98, 286, 1, 56, 121], [50, 100, 18

We have mapped recommendations for each user.
Now, for better understanding, we can map the names and then we can validate the accuracy of the model. Next we can compare it with the state of the art models.

Mapping movie by Name


In [ ]:
id_to_title = dict(zip(movies["movie_id"], movies["title"]))


In [ ]:
# recs: tensor of shape [num_users, k] with item *node indices*
# mapped = (recommended_items - num_users + 1).tolist()  # now a list of MovieLens item_ids


In [ ]:
def ids_to_titles(item_ids, id_to_title):
    return [id_to_title[i] for i in item_ids]

# Example: titles for first user (index 0 in recs)
# first_user_item_ids = mapped_recommendations[0]
# first_user_titles = ids_to_titles(first_user_item_ids, id_to_title)

# print(first_user_titles)


In [ ]:
# Convert node indices → MovieLens item_ids for all users
mapped_item_ids_per_user = (recommended_items - num_users + 1).tolist()


In [ ]:
#generate the titles of all recommended movies
# all_users_titles = [
#     [id_to_title[item_id] for item_id in user_item_ids]
#     for user_item_ids in mapped_item_ids_per_user
# ]
# print(all_users_titles)

In [ ]:
#get the id of all users and the titles of movies recommended
user_recs = {
    user_id + 1: [id_to_title[item_id] for item_id in user_item_ids]
    for user_id, user_item_ids in enumerate(mapped_item_ids_per_user)
}

print(user_recs)


{1: ['Star Wars (1977)', 'Fargo (1996)', 'Return of the Jedi (1983)', 'Contact (1997)', 'Godfather, The (1972)', 'Silence of the Lambs, The (1991)', 'English Patient, The (1996)', 'Toy Story (1995)', 'Pulp Fiction (1994)', 'Independence Day (ID4) (1996)'], 2: ['Star Wars (1977)', 'Fargo (1996)', 'Return of the Jedi (1983)', 'Contact (1997)', 'Godfather, The (1972)', 'Silence of the Lambs, The (1991)', 'English Patient, The (1996)', 'Toy Story (1995)', 'Pulp Fiction (1994)', 'Independence Day (ID4) (1996)'], 3: ['Star Wars (1977)', 'Fargo (1996)', 'Return of the Jedi (1983)', 'Contact (1997)', 'Godfather, The (1972)', 'Silence of the Lambs, The (1991)', 'English Patient, The (1996)', 'Toy Story (1995)', 'Pulp Fiction (1994)', 'Independence Day (ID4) (1996)'], 4: ['Star Wars (1977)', 'Fargo (1996)', 'Return of the Jedi (1983)', 'Contact (1997)', 'Godfather, The (1972)', 'Silence of the Lambs, The (1991)', 'English Patient, The (1996)', 'Toy Story (1995)', 'Pulp Fiction (1994)', 'Independ

In [ ]:
title_to_id = {title: movie_id for movie_id, title in id_to_title.items()}
print(title_to_id)

{'Toy Story (1995)': 1, 'GoldenEye (1995)': 2, 'Four Rooms (1995)': 3, 'Get Shorty (1995)': 4, 'Copycat (1995)': 5, 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)': 6, 'Twelve Monkeys (1995)': 7, 'Babe (1995)': 8, 'Dead Man Walking (1995)': 9, 'Richard III (1995)': 10, 'Seven (Se7en) (1995)': 11, 'Usual Suspects, The (1995)': 12, 'Mighty Aphrodite (1995)': 13, 'Postino, Il (1994)': 14, "Mr. Holland's Opus (1995)": 15, 'French Twist (Gazon maudit) (1995)': 16, 'From Dusk Till Dawn (1996)': 17, 'White Balloon, The (1995)': 18, "Antonia's Line (1995)": 19, 'Angels and Insects (1995)': 20, 'Muppet Treasure Island (1996)': 21, 'Braveheart (1995)': 22, 'Taxi Driver (1976)': 23, 'Rumble in the Bronx (1995)': 24, 'Birdcage, The (1996)': 25, 'Brothers McMullen, The (1995)': 26, 'Bad Boys (1995)': 27, 'Apollo 13 (1995)': 28, 'Batman Forever (1995)': 29, 'Belle de jour (1967)': 30, 'Crimson Tide (1995)': 31, 'Crumb (1994)': 32, 'Desperado (1995)': 33, 'Doom Generation, The (1995)': 34, 'Fr

In [ ]:
user_recs_ids = {}

for user_id, titles in user_recs.items():
    item_ids = [title_to_id[t] for t in titles]
    user_recs_ids[user_id] = item_ids


We have predictions mapped for each user. Now, we want to test the accuracy of the predictions by what the user predicted further. We use the base cases here.

In [ ]:
from collections import defaultdict

ground_truth = defaultdict(set)

for row in test_df.itertuples():
    user_id = row.user_id      # MovieLens user_id, 1..943
    item_id = row.item_id      # MovieLens movie_id, 1..1682
    ground_truth[user_id].add(item_id)


In [ ]:
import math

def recall_at_k(recs, truth, k):
    if not truth:
        return None  # user has no test data
    hits = len(set(recs[:k]) & truth)
    return hits / len(truth)

def precision_at_k(recs, truth, k):
    if k == 0:
        return 0.0
    hits = len(set(recs[:k]) & truth)
    return hits / k

def hitrate_at_k(recs, truth, k):
    return 1.0 if len(set(recs[:k]) & truth) > 0 else 0.0

def ndcg_at_k(recs, truth, k):
    dcg = 0.0
    for rank, item in enumerate(recs[:k], start=1):
        if item in truth:
            dcg += 1.0 / math.log2(rank + 1)

    ideal_hits = min(len(truth), k)
    if ideal_hits == 0:
        return 0.0

    idcg = sum(1.0 / math.log2(r + 1) for r in range(1, ideal_hits + 1))
    return dcg / idcg if idcg > 0 else 0.0


In [ ]:
K = 10

recall_list = []
precision_list = []
hitrate_list = []
ndcg_list = []

for user_id, rec_items in user_recs_ids.items():
    truth = ground_truth.get(user_id, set())

    if not truth:
        # skip users with no test interactions
        continue

    r = recall_at_k(rec_items, truth, K)
    p = precision_at_k(rec_items, truth, K)
    h = hitrate_at_k(rec_items, truth, K)
    n = ndcg_at_k(rec_items, truth, K)

    # recall can be None if truth is empty, but we already checked
    recall_list.append(r)
    precision_list.append(p)
    hitrate_list.append(h)
    ndcg_list.append(n)

print("Users evaluated:", len(recall_list))
print(f"Recall@{K}:   {sum(recall_list) / len(recall_list):.4f}")
print(f"Precision@{K}: {sum(precision_list) / len(precision_list):.4f}")
print(f"HitRate@{K}:   {sum(hitrate_list) / len(hitrate_list):.4f}")
print(f"NDCG@{K}:      {sum(ndcg_list) / len(ndcg_list):.4f}")


Users evaluated: 940
Recall@10:   0.0678
Precision@10: 0.1009
HitRate@10:   0.6138
NDCG@10:      0.1154


We are working with ratings as weight.

We use the following values for the model:
* optimizer = Adam,
* lr=1e-3,
* weight_decay=1e-4
* batch_size = 2048

epoch = 200

We get the following results:
Users evaluated: 940
* Recall@10:   0.0800
* Precision@10: 0.1061
* HitRate@10:   0.6500
* NDCG@10:      0.1219

Upon using ratings as training weights and increasing the epochs to 500, I got the following results:
* Users evaluated: 940
* Recall@10:   0.0710
* Precision@10: 0.1047
* HitRate@10:   0.6340
* NDCG@10:      0.1185

